In [7]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## Structured output parser

`StructuredOutputParser`는 여러 필드를 반환하고자 할 때 사용할 수 있습니다.


In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

응답 스키마를 정의합니다.


In [3]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(
        name="source",
        description="source used to answer the user's question, should be a website.",
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='answer', description="answer to the user's question", type='string'), ResponseSchema(name='source', description="source used to answer the user's question, should be a website.", type='string')])

이제 응답 형식에 대한 지침이 포함된 문자열을 가져와서 프롬프트에 삽입합니다.


In [28]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="사용자의 질문에 가능한 한 최선을 다해 답변해 주세요.\n{format_instructions}\n{question}. 한글로 답변해 주세요.",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```'}, template='사용자의 질문에 가능한 한 최선을 다해 답변해 주세요.\n{format_instructions}\n{question}. 한글로 답변해 주세요.')

Chat 모델을 사용하는 경우의 템플릿 정의


In [19]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            "사용자의 질문에 가능한 한 최선을 다해 답변해 주세요.\n{format_instructions}\n{question}"
        )
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

ChatPromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], template='사용자의 질문에 가능한 한 최선을 다해 답변해 주세요.\n{format_instructions}\n{question}'))])

모델을 불러옵니다.


In [29]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Chain 을 생성합니다.


In [30]:
chain = prompt | llm

In [38]:
response = chain.invoke({"question": "세종대왕의 업적에 대해서 알려줘"})

결과를 StructuredOutputParser를 통해 파싱 합니다.


In [39]:
output_parser.parse(response.content)

{'answer': '세종대왕(1397-1450)은 조선시대의 제4대 국왕으로, 한글을 창제한 업적으로 잘 알려져 있습니다. 그 외에도 세종대왕은 과학 기술, 문화 예술, 국방 등 다양한 분야에서 업적을 남겼습니다. 세종대왕은 한글 창제를 통해 국민들의 의사소통을 원활하게 하고, 국어 교육을 강화하여 국민들의 교양을 향상시키는 데 큰 역할을 했습니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}